In [ ]:
BiocManager::install("philr")
BiocManager::install("phyloseq")
BiocManager::install("microbiome")
install.packages("RColorBrewer")
install.packages("UpSetR")
install.packages("ggfortify")
install.packages("randomForest")
install.packages("rfUtilities")
install.packages("phytools")
install.packages("gridExtra")
install.packages("remotes")
install.packages('devtools')
install.packages("intergraph")
devtools::install_github('reptalex/phylofactor')
devtools::install_github("briatte/ggnet")
remotes::install_github("vmikk/metagMisc")
remotes::install_github("pmartinezarbizu/pairwiseAdonis/pairwiseAdonis")
remotes::install_github("gauravsk/ranacapa")
install.packages("ggdendro")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.14 (BiocManager 1.30.20), R 4.1.0 (2021-05-18)

Installing package(s) 'philr'

also installing the dependencies ‘gridGraphics’, ‘cachem’, ‘vctrs’, ‘stringi’, ‘ggplotify’, ‘patchwork’, ‘memoise’, ‘lazyeval’, ‘fastmatch’, ‘generics’, ‘igraph’, ‘quadprog’, ‘dplyr’, ‘purrr’, ‘stringr’, ‘tibble’, ‘tidyselect’, ‘isoband’, ‘mgcv’, ‘aplot’, ‘ggfun’, ‘yulab.utils’, ‘tidytree’, ‘treeio’, ‘ape’, ‘phangorn’, ‘tidyr’, ‘ggplot2’, ‘ggtree’


Warning message in install.packages(...):
“installation of package ‘stringi’ had non-zero exit status”


In [ ]:
library(philr, warn.conflicts = F, quietly = T)
library(RColorBrewer, warn.conflicts = F, quietly = T)
library(UpSetR, warn.conflicts = F, quietly = T)
library(ggfortify, warn.conflicts = F, quietly = T)
library(randomForest, warn.conflicts = F, quietly = T)
library(rfUtilities, warn.conflicts = F, quietly = T)
library(phytools, warn.conflicts = F, quietly = T)
library(phyloseq, warn.conflicts = F, quietly = T)
library(gridExtra, warn.conflicts = F, quietly = T)
library(microbiome, warn.conflicts = F, quietly = T)
library(phylofactor, warn.conflicts = F, quietly = T)
library(dplyr, warn.conflicts = F, quietly = T)
library(pairwiseAdonis, warn.conflicts = F, quietly = T)
library(ape, warn.conflicts = F, quietly = T)
library(metagMisc, warn.conflicts = F, quietly = T)
library(ranacapa, warn.conflicts = F, quietly = T)
library(MASS, warn.conflicts = F, quietly = T)
library(ggdendro, warn.conflicts = F, quietly = T)

In [ ]:
sessionInfo()

In [ ]:
seqtab <- read.table("../04-rpoC_processing/sequence_table.merged.txt", header=T, row.names=1)
tax <- read.table("taxonomy_bac.txt", header=F, row.names=1, sep="\t")
tree <- read.tree("RAxML_bestTree.ref.tre")
tree.root <- midpoint.root(tree)

In [ ]:
map <- read.table("../03-diff_abundance/map.txt", sep="\t", header=T, row.names=1)
notinmeta <- setdiff(colnames(seqtab), row.names(map))
notinraw <- setdiff(row.names(map), colnames(seqtab))
print("Samples found in ASV table but not in metadata:")
notinmeta
print("Samples found in metadata but not in sequencing table:")
notinraw

In [ ]:
ps.dat <- phyloseq(otu_table(seqtab, taxa_are_rows=T), sample_data(map), tax_table(as.matrix(tax)), tree.root)
ps.dat

In [ ]:
# compute prevalence dataframe
prevdf <- apply(X=otu_table(ps.dat), MARGIN=ifelse(taxa_are_rows(ps.dat), yes=1, no=2), FUN=function(x){sum(x>0)})
# add taxa and total read counts to dataframe
prevdf <- data.frame(Prevalence=prevdf, TotalAbundance=taxa_sums(ps.dat), tax_table(ps.dat))
# which phyla are comprised as mostly low prevalence ASVs?
lowprev <- ggplot(prevdf, aes(TotalAbundance, Prevalence, nsamples(ps.dat), color="V4")) + geom_hline(yintercept=0.05, alpha=0.5, linetype=2) + geom_point(size=2, alpha=0.7) + scale_x_log10() + xlab("Total Abundance") + ylab("Prevalence [Frac. Samples]") + facet_wrap(~V4) + theme(legend.position="none")
lowprev
pdf("totalabund_vs_prevalence.pdf")
lowprev
dev.off()
# kept asvs must be found in at least 1% of all samples 
ps.dat <- phyloseq_filter_prevalence(ps.dat, prev.trh=0.01)
ps.dat

In [ ]:
# filter out samples with fewer than 1000 reads (based on ASV rareness, this shouldn't be an issue)
ps.dat <- prune_samples(sample_sums(ps.dat) > 1000, ps.dat)
ps.dat

In [ ]:
# write filtered tables to file
write.table(as.data.frame(otu_table(ps.dat)), "sequence_table.filt.txt", sep="\t", row.names=T, col.names=T)
# write filtered taxonomy to file
write.table(as.data.frame(tax_table(ps.dat)), "taxonomy_bac.filt.txt", sep="\t", row.names=T, col.names=T)
# filtered metadata
write.table(as.data.frame(sample_data(ps.dat)), "map.filt.txt", sep="\t", row.names=T, col.names=T)

In [ ]:
# top phyla across samples (relative abundance data)
rel.abund <- transform_sample_counts(ps.dat, function(x) x/sum(x)) # get relative abundance
glom <- tax_glom(rel.abund, taxrank=rank_names(rel.abund)[3]) # collapse 
data <- psmelt(glom) # create dataframe from phyloseq object
data$V4 <- as.character(data$V4) # convert to character
data$V4[data$Abundance < 0.01] <- "< 1% abund" # rename low freq phyla
medians <- plyr::ddply(data, ~V4, function(x) c(median=median(x$Abundance)))
medians

In [ ]:
# most common genera
glom <- tax_glom(rel.abund, taxrank=rank_names(rel.abund)[8]) # collapse 
data <- psmelt(glom) # create dataframe from phyloseq object
data$V8 <- as.character(data$V8) # convert to character
data$V8[data$Abundance < 0.20] <- "< 20% abund" # rename low freq phyla
medians <- plyr::ddply(data, ~V8, function(x) c(median=median(x$Abundance)))
medians

In [ ]:
# phylum level figures
system("mkdir img")
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot by sample
taxbarsamp <- ggplot(data, aes(x=Sample, y=Abundance, fill=V4)) + geom_bar(aes(), stat="identity", position="stack") + theme_minimal() + theme(axis.text.x = element_text(angle = 90))
taxbarsamp
pdf("img/taxonomy_barchart.pdf")
taxbarsamp
dev.off()
# phyloseq group by tooth type
taxbargrp <- plot_bar(rel.abund, "V4", fill="V4", facet_grid="tooth_type") + geom_bar(aes(color=V4, fill=V4), stat="identity", position="stack")
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
pdf("img/tax_bar.tooth_type_by_sample.pdf")
taxbargrp
dev.off()

In [ ]:
# only including PD and PF samples for now
ps.dat <- subset_samples(ps.dat, tooth_type !="PE")
# distance based redundancy analysis
sample_data(ps.dat)$ads_nmol_min_mg_of_protein <- as.integer(sample_data(ps.dat)$ads_nmol_min_mg_of_protein)
sample_data(ps.dat)$log2_ads <- log2(sample_data(ps.dat)$ads_nmol_min_mg_of_protein)

ordcap <- ordinate(ps.dat, "CAP", "bray", ~ tooth_type + log2_ads)
# add dummy variable to sample data as plot_ordination cannot handle only one metadata column
sample_data(ps.dat)[ , 2] <- sample_data(ps.dat)[ ,1]
mid <- median(sample_data(ps.dat)$log2_ads)
pdf("img/capscale_plt.tooth_type.pdf") 
plot_ordination(ps.dat, ordcap, color="log2_ads", shape="tooth_type") + theme_minimal() + geom_point(size=4) + scale_color_gradient2(midpoint=mid, low="#91bfdb", mid="#ffffbf", high="#fc8d59")
dev.off()
plot_ordination(ps.dat, ordcap, color="log2_ads", shape="tooth_type") + theme_minimal() + geom_point(size=4) + scale_color_gradient2(midpoint=mid, low="#91bfdb", mid="#ffffbf", high="#fc8d59")

In [ ]:
# map labels to get sample position
plot_ordination(ps.dat, ordcap, shape="tooth_type", label = "novaseq_id") + 
    geom_text(mapping = aes(label = novaseq_id), size = 5, vjust = 1.5) + 
    theme(text = element_text(size = 10)) +
    scale_color_gradient2(midpoint=mid, low="#91bfdb", mid="#ffffbf", high="#fc8d59") + geom_point(size=2)

In [ ]:
head(sample_data(ps.dat))

plot_ordination(ps.dat, ordcap, shape="Smutans_cat", label = "novaseq_id") + 
    geom_text(mapping = aes(label = novaseq_id), size = 1, vjust = 1.5) + 
    theme(text = element_text(size = 10)) +
    scale_color_gradient2(midpoint=mid, low="#91bfdb", mid="#ffffbf", high="#fc8d59") + geom_point(size = 5)

In [ ]:

ordcap <- ordinate(ps.dat, "PCoA", "wunifrac", ~ tooth_type + log2_ads)
plot_ordination(ps.dat, ordcap, type = "split", shape="tooth_type", label = "novaseq_id", color = "V3") + 
    theme(text = element_text(size = 10)) + ylim(0.5,-0.5) + xlim(0.25,-0.25)

In [ ]:
# what's going on with top and bottom PD and PF?
write.table(sample_data(ps.dat), "map.txt", quote = F, sep = "\t")

In [ ]:
# is the difference between the amount of arginine detected significant between tooth groups?
wilcox.test(sample_data(ps.dat)[sample_data(ps.dat)$tooth_type == "PD",]$log2_ads, sample_data(ps.dat)[sample_data(ps.dat)$tooth_type == "PF",]$log2_ads)

In [ ]:
# what is up with low PF and high PD samples re: citrulline assay?
# samples are: UF49PF, UF33PF, UF40PFR, UF36PF, UF68PD
# sample_data(ps.dat)
x <- c("UF48PF", "UF33PF", "UF40PFR", "UF36PF", "UF68PD")
subset_lowhi_log2ADS <- phyloseq::prune_samples(x, ps.dat)
sample_data(subset_lowhi_log2ADS)
# do they have the taxa we are associating with ADS activity?
subset_lowhi_log2ADS_strep <- subset_taxa(subset_lowhi_log2ADS, V9=="Streptococcus_sanguinis" | V9=="Streptococcus_parasanguinis" | V9=="Streptococcus_oralis" | V9=="Limosilactobacillus_fermentum" | V9=="Leptotrichia_sp._oral_taxon_212")
glom <- tax_glom(subset_lowhi_log2ADS_strep, taxrank=rank_names(subset_lowhi_log2ADS_strep)[8])
# # what is proportion of species tagged in volcano plot in pd vs pf?
taxbargrp <- plot_bar(glom, "V9", fill="V9", facet_grid="tooth_type") + geom_bar(aes(color=V9, fill=V9), stat="identity", position="stack")
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
# by sample
taxbargrp <- plot_bar(glom, "V9", fill="V9", facet_grid="novaseq_id") + geom_bar(aes(color=V9, fill=V9), stat="identity", position="stack")
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
pdf("lowhi_ADS_prop_species.pdf")
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
dev.off()

In [ ]:
# with all six bacteria upregulated in disease
# do they have the taxa we are associating with ADS activity?
subset_lowhi_log2ADS_strep <- subset_taxa(subset_lowhi_log2ADS, V9=="Streptococcus_sanguinis" | V9=="Streptococcus_oralis" | V10=="Streptococcus_sp._oral_taxon_056" | V9=="Leptotrichia_sp._oral_taxon_212" | V9=="Leptotrichia_sp._oral_taxon_215" | V9=="Streptococcus_parasanguinis" | V10=="Streptococcus_intermedius" | V9=="Limosilactobacillus_fermentum" |  V9=="Parvimonas_micra" | V9=="Cryptobacterium_curtum")
glom <- tax_glom(subset_lowhi_log2ADS_strep, taxrank=rank_names(subset_lowhi_log2ADS_strep)[8])
# # what is proportion of species tagged in volcano plot in pd vs pf?
taxbargrp <- plot_bar(glom, "V9", facet_grid="tooth_type") + geom_bar(aes(color=V9, fill=V9), stat="identity", position="stack")
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

In [ ]:
# by sample
otu_table(glom)
taxbargrp <- plot_bar(glom, "V9", facet_grid="novaseq_id") + geom_bar(aes(), stat="identity", position="stack")
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
pdf("citrulline_oddballs_allspecies.pdf")
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
dev.off()

In [ ]:
# also want to know the proportion of S mutans and P acidifaciens across PD samples. Are there some that are dominanted by one or the other?
sub <- subset_taxa(ps.dat, V9=="Streptococcus_mutans" | V9=="Propionibacterium_acidifaciens")
glom <- tax_glom(sub, taxrank=rank_names(sub)[8])
# by sample
taxbargrp <- plot_bar(glom, "novaseq_id", fill="V9", facet_grid="V9") + geom_bar(aes(color=V9, fill=V9), stat="identity", position="stack") + facet_wrap(~tooth_type, ncol=1)
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) 
pdf("prop_smutans_pacidifaciens.pdf")
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
dev.off()

In [ ]:
# philr transform data and distance matrix to perform permanova statistics
philr.dat <- transform_sample_counts(ps.dat, function(x) x+1) # add pseudocount of one to ASVs to avoid log-ratios calculated from zero
is.rooted(phy_tree(philr.dat)) # check that tree is rooted
is.binary(phy_tree(philr.dat)) #check that multichotomies are resolved in tree
phy_tree(philr.dat) <- makeNodeLabel(phy_tree(philr.dat), method="number", prefix="n")                                    
asv.table <- otu_table(philr.dat)
tree <- phy_tree(philr.dat)
metadata <- sample_data(philr.dat)
tax <- tax_table(philr.dat)
philr.t <- philr(t(asv.table), tree, part.weights="enorm.x.gm.counts", ilr.weights="blw.sqrt")
# distance matrix
philr.dist <- dist(philr.t, method="euclidean")

In [ ]:
# permanova test
metadata <- as(sample_data(philr.dat), "data.frame")
adonis2(philr.dist ~ tooth_type, data=metadata)

In [ ]:
# marginal effect of ads amount after controlling for tooth type
adonis2(philr.dist ~ tooth_type + log2_ads, data=metadata)

In [ ]:
# beta dispersion test
dispr <- vegan::betadisper(philr.dist, phyloseq::sample_data(philr.dat)$tooth_type)
dispr
permutest(dispr)
boxplot(dispr)

In [ ]:
# alpha diversity
pdf("img/alpha_div.tooth_type.pdf")
plot_richness(ps.dat, measures=c("Observed", "Shannon"), x="tooth_type") + geom_boxplot() + theme_minimal() + geom_jitter()
dev.off()
plot_richness(ps.dat, measures=c("Observed", "Shannon"), x="tooth_type") + geom_boxplot() + theme_minimal() + geom_jitter()

In [ ]:
# install.packages("ggdist")
library(ggdist)
temp <- estimate_richness(subset_samples(ps.dat, tooth_type == "PD"))
temp$tooth_type <- "PD"
temp1 <- estimate_richness(subset_samples(ps.dat, tooth_type == "PF"))
temp1$tooth_type <- "PF"
alpha2 <- bind_rows(temp, temp1)

# raincloud plot
p <- ggplot(alpha2, aes(x = tooth_type, y = Shannon)) + 
  ggdist::stat_halfeye(
    adjust = .5, 
    width = .6, 
    .width = 0, 
    justification = -.3, 
    point_colour = NA) + 
  geom_boxplot(
    width = .25, 
    outlier.shape = NA
  ) +
  geom_point(
    size = 1.3,
    alpha = .5,
    position = position_jitter(
      seed = 1, width = .1
    )
  ) + 
    theme_minimal() +
  coord_cartesian(xlim = c(1.3, NA))
p
pdf("alphdiv_raincloud.pdf")
p
dev.off()

In [ ]:
# significance tests
wilcox.test(estimate_richness(subset_samples(ps.dat, tooth_type == "PD"))$Observed, estimate_richness(subset_samples(ps.dat, tooth_type == "PF"))$Observed)
wilcox.test(estimate_richness(subset_samples(ps.dat, tooth_type == "PD"))$Shannon, estimate_richness(subset_samples(ps.dat, tooth_type == "PF"))$Shannon)

In [ ]:
# proportion of different types of strep in pd vs pf
strep <- subset_taxa(ps.dat, V9=="Streptococcus_sanguinis")
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point() + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
    geom_rug()
geom_rug(sides ="bl")

In [ ]:
# proportion of different types of strep in pd vs pf
strep <- subset_taxa(ps.dat, V9=="Streptococcus_parasanguinis")
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point() + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
    geom_rug()
geom_rug(sides ="bl")

In [ ]:
# proportion of different types of strep in pd vs pf
strep <- subset_taxa(ps.dat, V9=="Streptococcus_oralis")
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point() + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
    geom_rug()
geom_rug(sides ="bl")

In [ ]:
# proportion of different types of strep in pd vs pf
strep <- subset_taxa(ps.dat, V9=="Streptococcus_gordonii")
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point() + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
    geom_rug()
geom_rug(sides ="bl")

In [ ]:
# proportion of different types of strep in pd vs pf
strep <- subset_taxa(ps.dat, V9=="Streptococcus_anginosus_group") # includes both constellatus and aginosus
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point() + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
    geom_rug()
geom_rug(sides ="bl")

In [ ]:
strep <- subset_taxa(ps.dat, V8=="Streptococcus")
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
# what is proportion of different strep species in pd vs pf?
taxbargrp <- plot_bar(glom, "V9", fill="V9", facet_grid="caries_status") + geom_bar(aes(color=V9, fill=V9), stat="identity", position="stack")
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

tax_table(subset_taxa(ps.dat, V10=="Streptococcus_constellatus"))

In [ ]:
strep <- subset_taxa(ps.dat, V9=="Streptococcus_sanguinis" | V9=="Streptococcus_parasanguinis" | V9=="Streptococcus_oralis" | V9=="Limosilactobacillus_fermentum" | V9=="Leptotrichia_sp._oral_taxon_212")
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
# what is proportion of species tagged in volcano plot in pd vs pf?
taxbargrp <- plot_bar(glom, "V9", fill="V9", facet_grid="tooth_type") + geom_bar(aes(color=V9, fill=V9), stat="identity", position="stack")
pdf("bar_matching_volcano_plot.pdf")
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
dev.off()
taxbargrp + theme_minimal() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

In [ ]:
# differential abundance data
library(DESeq2)

In [ ]:
sessionInfo()

In [ ]:
head(sample_data(ps.dat))

In [ ]:
difabund <- phyloseq_to_deseq2(ps.dat, ~ tooth_type)

gm_mean = function(x, na.rm=TRUE){
  exp(sum(log(x[x > 0]), na.rm=na.rm) / length(x))
}

geoMeans <- apply(counts(difabund), 1, gm_mean)
difabund <- estimateSizeFactors(difabund, geoMeans = geoMeans)
res <- DESeq(difabund, fitType="local")

sigtab <- results(res)
sigtab <- sigtab[order(sigtab$padj, na.last = NA),]
sigtab <- sigtab[(sigtab$padj < 0.05),]
sigtab <- cbind(as(sigtab, "data.frame"), as(tax_table(ps.dat)[rownames(sigtab),], "matrix"))
sigtab


In [ ]:
theme_set(theme_bw())
scale_fill_discrete <- function(palname = "Set1", ...) {
    scale_fill_brewer(palette = palname, ...)
}
# Phylum order
x = tapply(sigtab$log2FoldChange, sigtab$V10, function(x) max(x))
x = sort(x, TRUE)
sigtab$V10 = factor(as.character(sigtab$V10), levels=names(x))
# Genus order
x = tapply(sigtab$log2FoldChange, sigtab$V10, function(x) max(x))
x = sort(x, TRUE)
sigtab$V10 = factor(as.character(sigtab$V10), levels=names(x))
ggplot(sigtab, aes(x=V10, y=log2FoldChange, color=V8)) + geom_point(size=6) + 
  theme(axis.text.x = element_text(angle = -90, hjust = 0, vjust=0.5))
pdf("diff_abundance.pdf")
ggplot(sigtab, aes(x=V10, y=log2FoldChange, color=V8)) + geom_point(size=6) + 
  theme(axis.text.x = element_text(angle = -90, hjust = 0, vjust=0.5))
dev.off()

In [ ]:
sigtab <- results(res)
sigtab <- sigtab[order(sigtab$padj, na.last = NA),]
sigtab <- sigtab[(sigtab$padj < 0.05),]
sigtab <- cbind(as(sigtab, "data.frame"), as(tax_table(ps.dat)[rownames(sigtab),], "matrix"))
sigtab

In [ ]:
allres <- results(res, independentFiltering = FALSE)
allres <- allres[order(allres$padj, na.last = NA),]
allres <- cbind(as(allres, "data.frame"), as(tax_table(ps.dat)[rownames(allres),], "matrix"))
write.table(allres, "full_deseq_results.rpoc.txt")

In [ ]:
###### FIGURE 2: DESEQ FOR RPOC COLLAPSED AT SPECIES LEVEL
# now need to summarize at species level, re run deseq
# since there are all sorts of levels in the ncbi taxonomy, 
# first glom down to lower level, 
# save and summarize manually in excel, 
# re read in as sequence table to be made into phyloseq object
glomtax <- tax_glom(ps.dat, taxrank=rank_names(ps.dat)[9]) # collapse 
glomtax
# as.vector(tax_table(glomtax)[,9])
write.table(otu_table(glomtax), "collapsed_ASV_table.txt", sep = "\t", quote = F)
write.table(tax_table(glomtax), "collapsed_taxonomy_table.txt", sep = "\t", quote = F)

In [ ]:
# read taxonomy and ASV table back in, make new phyloseq object
seqtab <- read.table("collapsed_ASV_table_FIXED.txt", header=T, row.names=1)
tax <- read.table("collapsed_taxonomy_table_FIXED.txt", header=F, row.names=1, sep="\t")
ps.dat.sp <- phyloseq(otu_table(seqtab, taxa_are_rows=T), sample_data(map), tax_table(as.matrix(tax)), tree.root)
ps.dat.sp

In [ ]:
difabund <- phyloseq_to_deseq2(ps.dat.sp, ~ tooth_type)
difabund
geoMeans <- apply(counts(difabund), 1, gm_mean)
difabund <- estimateSizeFactors(difabund, geoMeans = geoMeans)
res <- DESeq(difabund, fitType="local")

sigtab <- results(res)
sigtab <- sigtab[order(sigtab$padj, na.last = NA),]
sigtab <- sigtab[(sigtab$padj < 0.05),]
sigtab <- cbind(as(sigtab, "data.frame"), as(tax_table(ps.dat)[rownames(sigtab),], "matrix"))
sigtab

In [ ]:
theme_set(theme_bw())
scale_fill_discrete <- function(palname = "Set1", ...) {
    scale_fill_brewer(palette = palname, ...)
}
# Phylum order
x = tapply(sigtab$log2FoldChange, sigtab$V10, function(x) max(x))
x = sort(x, TRUE)
sigtab$V10 = factor(as.character(sigtab$V10), levels=names(x))
# Genus order
x = tapply(sigtab$log2FoldChange, sigtab$V10, function(x) max(x))
x = sort(x, TRUE)
sigtab$V10 = factor(as.character(sigtab$V10), levels=names(x))
ggplot(sigtab, aes(x=V10, y=log2FoldChange, color=V8)) + geom_point(size=6) + 
  theme(axis.text.x = element_text(angle = -90, hjust = 0, vjust=0.5)) + theme(legend.position = "none")
pdf("diff_abundance.taxglom.pdf")
ggplot(sigtab, aes(x=V10, y=log2FoldChange, color=V8)) + geom_point(size=6) + 
  theme(axis.text.x = element_text(angle = -90, hjust = 0, vjust=0.5))
dev.off()

In [ ]:
allres <- results(res, independentFiltering = FALSE)
allres <- allres[order(allres$padj, na.last = NA),]
allres <- cbind(as(allres, "data.frame"), as(tax_table(ps.dat)[rownames(allres),], "matrix"))
write.table(allres, "full_deseq_results.collapse_species_rpoc.txt")

In [ ]:
# want to get fold changes per sample from deseq results
mcols(mcols(res), use.names=TRUE)$description
head(mcols(res))

In [ ]:
# log fold change figure and correlation
lfc <- read.table("logFC_for_plot.txt", header=T)
# head(lfc)
# reorder
lfc$Taxon <- with(lfc, reorder(Taxon, lfc2, max))

ggplot(lfc, aes(x=Taxon, y=lfc2, group=source)) + 
    geom_bar(stat="identity", aes(fill=factor(source)), position=position_dodge(width=0.9)) +
    coord_flip()

pdf("log2FC_plot_DNA_v_RNA.pdf")
ggplot(lfc, aes(x=Taxon, y=lfc2, group=source)) + 
    geom_bar(stat="identity", aes(fill=factor(source)), position=position_dodge(width=0.9)) +
    coord_flip()
dev.off()


In [ ]:
# correlation between lot2 fold change
temp <- reshape(lfc, idvar="Taxon", timevar="source", direction="wide")

ggplot(temp, aes(x=lfc2.DNA, y=lfc2.RNA)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) 

pdf("dna_vs_rna_corplot.pdf")
ggplot(temp, aes(x=lfc2.DNA, y=lfc2.RNA)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE)
dev.off()

cor.test(temp$lfc2.DNA, temp$lfc2.RNA)

In [ ]:
# correlation with ADS activity (citrulline production) of specific species
head(sample_data(ps.dat))

In [ ]:
# different taxa affect on citrulline production
strep <- subset_taxa(ps.dat, V9=="Streptococcus_sanguinis") 
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")

pdf("correlation_citrulline_ssanguinis.pdf")
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")
dev.off()

temp <- cbind(log2(data$Abundance), data$log2_ads)
temp[is.na(temp) | temp=="-Inf"] <- NA
temp <- as.data.frame(temp)
summary(lm(V1~V2, data=temp))

strep <- subset_taxa(ps.dat, V9=="Leptotrichia_sp._oral_taxon_215") 
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")

pdf("correlation_citrulline_lepto215.pdf")
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")
dev.off()

strep <- subset_taxa(ps.dat, V9=="Leptotrichia_sp._oral_taxon_212") 
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")

pdf("correlation_citrulline_lepto212.pdf")
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")
dev.off()

temp <- cbind(log2(data$Abundance), data$log2_ads)
temp[is.na(temp) | temp=="-Inf"] <- NA
temp <- as.data.frame(temp)
summary(lm(V1~V2, data=temp))

strep <- subset_taxa(ps.dat, V9=="Streptococcus_mutans") 
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")

pdf("correlation_citrulline_smutans.pdf")
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")
dev.off()

temp <- cbind(log2(data$Abundance), data$log2_ads)
temp[is.na(temp) | temp=="-Inf"] <- NA
temp <- as.data.frame(temp)
summary(lm(V1~V2, data=temp))

strep <- subset_taxa(ps.dat, V9=="Streptococcus_oralis") 
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")

pdf("correlation_citrulline_soralis.pdf")
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")
dev.off()

temp <- cbind(log2(data$Abundance), data$log2_ads)
temp[is.na(temp) | temp=="-Inf"] <- NA
temp <- as.data.frame(temp)
summary(lm(V1~V2, data=temp))

strep <- subset_taxa(ps.dat, V9=="Streptococcus_parasanguinis") 
glom <- tax_glom(strep, taxrank=rank_names(strep)[8])
data <- psmelt(glom) # create dataframe from phyloseq object
data$Sample <- factor(data$Sample, levels=unique(data$Sample))
# plot
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")

pdf("correlation_citrulline_sparasanguinis.pdf")
ggplot(data, aes(x=log2(Abundance), y=log2_ads, color=tooth_type)) + 
    geom_point(size = 4) + 
    theme_minimal() + 
    geom_smooth(method=lm, se=FALSE, fullrange=TRUE) +
#     geom_rug(outside = TRUE, sides = "tr", linewidth = 10) +
    coord_cartesian(clip = "off")
dev.off()

temp <- cbind(log2(data$Abundance), data$log2_ads)
temp[is.na(temp) | temp=="-Inf"] <- NA
temp <- as.data.frame(temp)
summary(lm(V1~V2, data=temp))

In [ ]:
# now need to get log abundance and count histograms for each of the ADS species in log fold change plot
abct <- read.table("abundance_count.txt", header=T)
head(abct)

In [ ]:
# reorder
abct$Taxon <- with(abct, reorder(Taxon, order, max))
# head(abct)
ggplot(abct, aes(x=Taxon, y=log10_sum)) + 
    geom_bar(stat="identity") +
    coord_flip() + theme_minimal()

pdf("abundance_barplot.pdf")
ggplot(abct, aes(x=Taxon, y=log10_sum)) + 
    geom_bar(stat="identity") +
    coord_flip() + theme_minimal()
dev.off()

ggplot(abct, aes(x=Taxon, y=Prop)) + 
    geom_bar(stat="identity") +
    coord_flip() + theme_minimal()

pdf("proportion_barplot.pdf")
ggplot(abct, aes(x=Taxon, y=Prop)) + 
    geom_bar(stat="identity") +
    coord_flip() + theme_minimal()
dev.off()

In [ ]:
#smutans cat
# rel.abund
# head(sample_data(rel.abund))
test <- subset_taxa(rel.abund, V9 == "Streptococcus_oralis" | V9 == "Streptococcus_sanguinis")
p <- plot_bar(test, fill = "V9", facet_grid = Smutans_cat~tooth_type) + theme_minimal()
p + geom_bar(aes(color=V9, fill=V9), stat="identity", position="stack")
pdf("relabund_soralis_sanguinis_smutans_lohi_pdvspf.pdf")
p + geom_bar(aes(color=V9, fill=V9), stat="identity", position="stack")
dev.off()